#### 草稿本


In [2]:
# TEST CONNECTION AND IMPORT LIBRARY
import dmPython

conn = dmPython.connect(
                user="SYSDBA",
                password="lamweiyin",
                server="36.140.31.145",
                port=31826,
                autoCommit=True)
cursor = conn.cursor()

cursor.execute('select username from dba_users')

results = cursor.fetchall()
for row in results:
    print(row)
conn.close()

('SYSAUDITOR',)
('WEIYIN',)
('SYSSSO',)
('SYSDBA',)
('SYS',)


In [51]:
# CREATE TABLESPACES AND USERS

conn = dmPython.connect(
                user="SYSDBA",
                password="lamweiyin",
                server="36.140.31.145",
                port=31826,
                autoCommit=True)
cursor = conn.cursor()

uname = 'weiyin'
passwd = 'lamweiyin'

cursor.execute(f"create tablespace {uname} datafile '{uname}.dbf' size 256 autoextend on maxsize 10240")
cursor.execute(f'create user {uname} identified by "{passwd}" default tablespace {uname} default index tablespace {uname}')
cursor.execute(f'grant "RESOURCE", "PUBLIC", "DBA", "VTI" to {uname}')

conn.close()

In [52]:
# CREATE TABLES

conn = dmPython.connect(
                user="weiyin",
                password="lamweiyin",
                server="36.140.31.145",
                port=31826,
                autoCommit=True)
cursor = conn.cursor()

cursor.execute(
    """create table 物流公司 (
    公司名称 varchar(100) not null unique,
    客户编号 varchar(50) not null,
    联系人 varchar(50),
    电话 varchar(20),
    省市区 varchar(100),
    primary key(客户编号))"""
)
cursor.execute(
    """create table 客户信息 (
    客户名称 varchar(50) not null,
    客户编号 varchar(20) not null,
    手机号 varchar(20) not null unique,
    省市区 varchar(100),
    primary key(客户编号))"""
)
cursor.execute(
    """create table 物流信息 (
    提单号 varchar(50) not null,
    货主名称 varchar(50) not null,
    货主代码 varchar(20) not null foreign key references 客户信息(客户编号),
    物流公司_货代 varchar(100) not null foreign key references 物流公司(公司名称),
    集装箱箱号 varchar(50) not null,
    货物名称 varchar(50) not null,
    货重_吨 varchar(50) not null,
    primary key(提单号))"""
)
cursor.execute(
    """create table 集装箱动态 (
    堆存港口 varchar(100) not null,
    集装箱箱号 varchar(50) not null,
    箱尺寸_TEU varchar(50),
    提单号 varchar(100) not null foreign key references 物流信息(提单号),
    堆场位置 varchar(100) not null,
    操作 varchar(50) not null,
    操作日期 varchar(50) not null,
    primary key(集装箱箱号, 提单号, 操作))"""
)
cursor.execute(
    """create table 装货表 (
    船公司 varchar(100),
    船名称 varchar(100),
    作业开始时间 varchar(100),
    作业结束时间 varchar(100),
    始发时间 varchar(100),
    到达时间 varchar(100),
    作业港口 varchar(100),
    提单号 varchar(100) not null foreign key references 物流信息(提单号),
    集装箱箱号 varchar(100),
    箱尺寸_TEU varchar(100),
    启运地 varchar(100),
    目的地 varchar(100),
    primary key(提单号))"""
)
cursor.execute(
    """create table 卸货表 (
    船公司 varchar(100),
    船名称 varchar(100),
    作业开始时间 varchar(100),
    作业结束时间 varchar(100),
    始发时间 varchar(100),
    到达时间 varchar(100),
    作业港口 varchar(100),
    提单号 varchar(100) not null foreign key references 物流信息(提单号),
    集装箱箱号 varchar(100),
    箱尺寸_TEU varchar(100),
    启运地 varchar(100),
    目的地 varchar(100),
    primary key(提单号))"""
)
cursor.execute(
    """create table 假客户信息 (
    客户名称 varchar(50),
    客户编号 varchar(20),
    手机号 varchar(20),
    省市区 varchar(100))"""
)
cursor.execute("create table 记录信息 (表名 varchar(100), 记录个数 int)")
for table in ["物流公司", "客户信息", "物流信息", "集装箱动态", "装货表", "卸货表"]:
    cursor.execute(f"insert into 记录信息 values ('{table}', 0)")

conn.close()

In [53]:
# LIST TABLES

conn = dmPython.connect(
                user="weiyin",
                password="lamweiyin",
                server="36.140.31.145",
                port=31826,
                autoCommit=True)
cursor = conn.cursor()

uname = 'WEIYIN'
cursor.execute(f"""select NAME
               from sysobjects
               where \"SUBTYPE$\"='UTAB'
               AND SCHID=(SELECT ID
               FROM sysobjects
               WHERE NAME='{uname}' AND TYPE$='SCH')""")

results = cursor.fetchall()
if len(results) == 0:
    print('Empty table')
for row in results:
    print(row)
conn.close()

('集装箱动态',)
('客户信息',)
('物流公司',)
('物流信息',)
('卸货表',)
('装货表',)


In [49]:
# DELETE USER

conn = dmPython.connect(
                user="SYSDBA",
                password="lamweiyin",
                server="36.140.31.145",
                port=31826,
                autoCommit=True)
cursor = conn.cursor()

uname = "weiyin"
cursor.execute(f"drop user {uname} cascade")
cursor.execute(f"drop tablespace {uname}")

conn.close()

In [90]:
# 港口吞吐量分析
conn = dmPython.connect(
                user="weiyin",
                password="lamweiyin",
                server="36.140.31.145",
                port=31826,
                autoCommit=True)
cursor = conn.cursor()

cursor.execute(
    """select 堆存港口, count(*) as 吞吐量
    from 集装箱动态
    group by 堆存港口
    order by 吞吐量 desc"""
)
res = {}
for item in cursor.fetchall():
    res[item[0]] = item[1]

conn.close()
res

{'芜湖港': 7368,
 '镇江港': 7290,
 '泰州港': 7228,
 '太仓港': 7038,
 '南通港': 7015,
 '铜陵港': 6947,
 '马鞍山港': 5948,
 '苏州港': 5897,
 '杭州港': 4891}

In [ ]:
# 港口不同类型货物吞吐量趋势
conn = dmPython.connect(
                user="weiyin",
                password="lamweiyin",
                server="36.140.31.145",
                port=31826,
                autoCommit=True)
cursor = conn.cursor()

cursor.execute(
    """select 货物名称, count(货重_吨) as 客户量, sum(货重_吨) as 总量
    from 物流信息
    group by 货物名称
    order 总量 desc, 客户量 desc"""
)
res = {}
for item in cursor.fetchall():
    res[item[0]] = item[1]

conn.close()
res
